In [1]:
!wget https://raw.githubusercontent.com/davidADSP/Generative_Deep_Learning_2nd_Edition/main/notebooks/utils.py

--2023-05-05 04:49:50--  https://raw.githubusercontent.com/davidADSP/Generative_Deep_Learning_2nd_Edition/main/notebooks/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 771 [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]     771  --.-KB/s    in 0s      

2023-05-05 04:49:50 (30.7 MB/s) - ‘utils.py’ saved [771/771]



In [2]:
import numpy as np

import tensorflow as tf
from tensorflow.keras import (
    datasets,
    layers,
    models,
    optimizers,
    activations,
    metrics,
    callbacks,
)

from utils import display, sample_batch
import random

In [3]:
IMAGE_SIZE = 32
CHANNELS = 1
STEP_SIZE = 10
STEPS = 60
NOISE = 0.005
ALPHA = 0.1
GRADIENT_CLIP = 0.03
BATCH_SIZE = 128
BUFFER_SIZE = 8192
LEARNING_RATE = 0.0001
EPOCHS = 60

In [4]:
# Load the data
(x_train, _), (x_test, _) = datasets.mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


In [5]:
# Preporcess the data
def preprocess(imgs):
  """Normalize and reshape the images"""
  imgs = (imgs.astype("float32") - 127.5) / 127.5
  imgs = np.pad(imgs, ((0, 0), (2, 2), (2, 2)), constant_values=-1.0)
  imgs = np.expand_dims(imgs, -1)
  return imgs

x_train = preprocess(x_train)
x_test = preprocess(x_test)

In [6]:
x_train = tf.data.Dataset.from_tensor_slices(x_train).batch(BATCH_SIZE)
x_test = tf.data.Dataset.from_tensor_slices(x_test).batch(BATCH_SIZE)

## The Energy Function
The energy function $E_{\theta}(x)$ is a neural network with parameters $\theta$ that can transform an input image $x$ into a scalar value.

The network is a set of stacked `Conv2D` layers that gradually reduce the size of the image while increasing the number of channels. This final layer is a single fully connected unit with linear activation, so the network can output values in the range $(-∞, ∞)$.

In [7]:
ebm_input = layers.Input(shape=(32, 32, 1))
x = layers.Conv2D(16, kernel_size=5, strides=2, padding="same", activation=activations.swish)(ebm_input)
x = layers.Conv2D(32, kernel_size=3, strides=2, padding="same", activation=activations.swish)(x)
x = layers.Conv2D(64, kernel_size=3, strides=2, padding="same", activation=activations.swish)(x)
x = layers.Conv2D(64, kernel_size=3, strides=2, padding="same", activation=activations.swish)(x)
x = layers.Flatten()(x)
x = layers.Dense(64, activation=activations.swish)(x)
ebm_output = layers.Dense(1)(x)
model = models.Model(ebm_input, ebm_output)

##The Langevin sampling function

In [8]:
def generate_samples(model, inp_imgs, steps, step_size, noise, return_img_per_step=False):
  imgs_per_step = []
  for _ in range(steps):
    inp_imgs += tf.random.normal(inp_imgs.shape, mean=0, stddev=noise)
    inp_imgs = tf.clip_by_value(inp_imgs, -1.0, 1.0)
    with tf.GradientTape() as tape:
      tape.watch(inp_imgs)
      out_score = model(inp_imgs)
    grads = tape.gradient(out_score, inp_imgs)
    grads = tf.clip_by_value(grads, -GRADIENT_CLIP, GRADIENT_CLIP)
    inp_imgs += step_size * grads
    inp_imgs = tf.clip_by_value(inp_imgs, -1.0, 1.0)
    if return_img_per_step:
      imgs_per_step.append(inp_imgs)
  if return_img_per_step:
    return tf.stack(imgs_per_step, axis=0)
  else:
    return inp_imgs

In [9]:
class Buffer:
  def __init__(self, model):
    super().__init__()
    self.model = model
    self.examples = [
        tf.random.uniform(shape=(1, IMAGE_SIZE, IMAGE_SIZE, CHANNELS)) * 2 -1
        for _ in range(128)
    ]

  def sample_new_exmps(self, steps, step_size, noise):
    n_new = np.random.binomial(BATCH_SIZE, 0.05)
    rand_imgs = (
        tf.random.uniform((n_new, IMAGE_SIZE, IMAGE_SIZE, CHANNELS)) * 2 - 1
    )
    old_imgs = tf.concat(
        random.choices(self.examples, k=BATCH_SIZE - n_new), axis=0
    )
    inp_imgs = tf.concat([rand_imgs, old_imgs], axis=0)
    inp_imgs = generate_samples(
        self.model, inp_imgs, steps=steps, step_size=step_size, noise=noise
    )
    self.examples = tf.split(inp_imgs, BATCH_SIZE, axis=0) + self.examples
    self.examples = self.examples[:8192]
    return inp_imgs

In [ ]:
class EBM(models.Model):
  def __init__(self):
    super(EBM, self).__init__()
    self.model = model
    self.buffer = Buffer(self.model)
    self.alpha = ALPHA
    self.loss_metric = metrics.Mean(name="loss")
    self.reg_loss_metric = metrics.Mean(name="reg")
    self.cdiv_loss_metric = metrics.Mean(name="cdiv")  
    self.real_out_metric = metrics.Mean(name="real")
    self.fake_out_metric = metrics.Mean(name="fake")

  @property
  def metrics(self):
    return [
        self.loss_metric,
        self.reg_loss_metric,
        self.cdiv_loss_metric,
        self.real_out_metric,
        self.fake_out_metric
    ]

  def train_step(self, real_imgs):
    real_imgs += tf.random.normal(
        shape=tf.shape(real_imgs), mean=0, stddev=NOISE
    )
    real_imgs = tf.clip_by_value(real_imgs, -1.0, 1.0)
    fake_imgs = self.buffer.sample_new_exmps(
        steps=STEPS, step_size=STEP_SIZE, noise=NOISE
    )
    inp_imgs = tf.concat([real_imgs, fake_imgs], axis=0)
    with tf.GradientTape() as training_tape:
      real_out, fake_out = tf.split(self.model(inp_imgs), 2, axis=0)
      cdiv_loss = tf.reduce_mean(fake_out, axis=0) - tf.reduce_mean(
          real_out, axis=0
      )
      reg_loss = self.alpha * tf.reduce_mean(
          real_out**2 + fake_out**2, axis=0
      )
      loss = cdiv_loss + reg_loss 
    grads = training_tape.gradient(loss, self.model.trainable_variables)
    self.optimizer.apply_gradients(
        zip(grads, self.model.trainable_variables)
    )
    self.loss_metric.update_state(loss)
    self.reg_loss_metric.update_state(reg_loss)
    self.cdiv_loss_metric.update_state(cdiv_loss)
    self.real_out_metric.update_state(tf.reduce_mean(real_out, axis = 0))
    self.fake_out_metric.update_state(tf.reduce_mean(fake_out, axis = 0))
    return {m.name: m.result() for m in self.metrics}

  def test_step(self, real_imgs): 
      batch_size = real_imgs.shape[0]
      fake_imgs = tf.random.uniform((batch_size, 32, 32, 1)) * 2 - 1
      inp_imgs = tf.concat([real_imgs, fake_imgs], axis=0)
      real_out, fake_out = tf.split(self.model(inp_imgs), 2, axis=0)
      cdiv = tf.reduce_mean(fake_out, axis = 0) - tf.reduce_mean(
          real_out, axis = 0
      )
      self.cdiv_loss_metric.update_state(cdiv)
      self.real_out_metric.update_state(tf.reduce_mean(real_out, axis = 0))
      self.fake_out_metric.update_state(tf.reduce_mean(fake_out, axis = 0))
      return {m.name: m.result() for m in self.metrics[2:]}

ebm = EBM()
ebm.compile(optimizer=optimizers.Adam(learning_rate=0.0001), run_eagerly=True)
ebm.fit(x_train, epochs=60, validation_data = x_test,)

Epoch 1/60
